# **MONETARY SMURFING ANALYTICS**

## **DATASET MOUNTING**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **DATA PREPROCESSING 1**

In [4]:
import pandas as pd
import numpy as np
import json
import csv
import logging
from random import randint

data_path = "/content/drive/MyDrive/miniproject 2/project/data.csv"

# logging.basicConfig(filename='logs/model_development.txt',
# 					filemode='a',
# 					format='%(asctime)s %(message)s',force=True,
# 					datefmt="%Y-%m-%d %H:%M:%S")

logging.warning("DATA PREPROCESSING 1 STAGE")

logging.warning("Reading Dataset...")

X = pd.read_csv(data_path)
X = X.to_numpy()

logging.warning("Read Dataset")

nameOrigCol = 3
nameDestCol = 6
nameOrig = []
nameDest = []
nameCount = {}
namesWithMoreThanOneOccurrence = []

logging.warning("Checking Each Person's Transactions Count...")

for name in X[:,nameOrigCol]:
	if nameCount.get(name,-1) == -1:
		nameOrig.append(name)

		nameCount[name] = 1

	else:
		nameCount[name] += 1
		namesWithMoreThanOneOccurrence.append(name)

for name in X[:,nameDestCol]:
	if nameCount.get(name,-1) == -1:
		nameDest.append(name)

		nameCount[name] = 1

	else:
		nameCount[name] += 1
		namesWithMoreThanOneOccurrence.append(name)

logging.warning("Count Identification Done")

logging.warning("Calculating Median ...")

countArr = []
count = 0
for attr, value in nameCount.items():
	if value>40:
		countArr.append(value)
		count += 1
median = np.median(countArr)

logging.warning(f"Median : {median}")

logging.warning("Filtering Data Based on Transactions Count...")
csv_golden_data = []

for i in range(X.shape[0]):
	if nameCount.get(X[i,3],-1) > 40 or nameCount.get(X[i,6],-1) > 40:
		csv_golden_data.append(X[i,:])

logging.warning("Filtering Done")

logging.warning("Storing Filtered Data in data_processed folder...")

with open("/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(csv_golden_data)

logging.warning("Data is Stored")





In [5]:
data_path = "/content/drive/MyDrive/miniproject 2/project/data.csv"
filtereddata = pd.read_csv(data_path)
filtereddata.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [6]:
data_path = "/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data.csv"
filtereddata = pd.read_csv(data_path)
filtereddata.head()

,1,TRANSFER,181.0,C1305486145,181.0.1,0.0,C553264065,0.0.1,0.0.2,1.1,0
0,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.00,1,0
1,1,DEBIT,9644.94,C1900366749,4465.0,0.00,C997608398,10845.0,157982.12,0,0
2,1,CASH_OUT,229133.94,C905080434,15325.0,0.00,C476402209,5083.0,51513.44,0,0
3,1,TRANSFER,215310.30,C1670993182,705.0,0.00,C1100439041,22425.0,0.00,0,0
4,1,DEBIT,9302.79,C1566511282,11299.0,1996.21,C1973538135,29832.0,16896.70,0,0


## **DATA PREPROCESSING 2**

In [7]:
import pandas
import json
import numpy as np
import csv
import logging
from random import randint

data_path = "/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data.csv"

# logging.basicConfig(filename='logs/model_development.txt',
# 					filemode='a',
# 					format='%(asctime)s %(message)s',
# 					datefmt="%Y-%m-%d %H:%M:%S")

logging.warning("----------")
logging.warning("DATA PREPROCESSING 2 STAGE")

logging.warning("Reading Preprocessed 1 dataset...")

X = pandas.read_csv(data_path)
X = X.to_numpy()

logging.warning("Read Preprocessed 1 dataset...")

csv_dataset_primary = []
step = 0
trans_type = 1
amount = 2
nameOrig = 3
oldbalanceOrg = 4
nameDest = 6
oldbalanceDest = 7
accountType = 8
isFraud = 9
isFlaggedFraud = 10

logging.warning("Changing Labels of Type Column ...")

transfer = ["WIRE_IN", "WIRE_OUT"]
for i in range(X.shape[0]):
	arr = []
	arr.append(X[i,step])
	if X[i,trans_type] =="PAYMENT":
		arr.append("CREDIT")
	elif X[i,trans_type] =="TRANSFER":
		arr.append(transfer[randint(0,1)])
	else:
		arr.append(X[i,trans_type])
	arr.append(X[i,amount])
	arr.append(X[i,nameOrig])
	arr.append(X[i,oldbalanceOrg])
	arr.append(X[i,nameDest])
	arr.append(X[i,oldbalanceDest])
	if X[i,trans_type] == "TRANSFER":
		arr.append("FOREIGN")
	else:
		arr.append("DOMESTIC")

	arr.append(X[i,isFraud])
	arr.append(X[i,isFlaggedFraud])

	csv_dataset_primary.append(arr)

logging.warning("Changing Labels Done")
logging.warning("Storing Data in Data_processed Folder...")


columns=['step','trans_type','amount','nameOrig','oldbalanceOrg',
        'nameDest','oldbalanceDest','accountType','isFraud','isFlaggedFraud']

data_primary = pandas.DataFrame(csv_dataset_primary, columns=columns)

data_primary.to_csv('/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data_2.csv', index=False)

logging.warning("Storing Data Done")





In [8]:
data_path = "/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data_2.csv"
filtereddata = pd.read_csv(data_path)
filtereddata.head()

,step,trans_type,amount,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest,accountType,isFraud,isFlaggedFraud
0,1,CASH_OUT,181.00,C840083671,181.0,C38997010,21182.0,DOMESTIC,1,0
1,1,DEBIT,9644.94,C1900366749,4465.0,C997608398,10845.0,DOMESTIC,0,0
2,1,CASH_OUT,229133.94,C905080434,15325.0,C476402209,5083.0,DOMESTIC,0,0
3,1,WIRE_IN,215310.30,C1670993182,705.0,C1100439041,22425.0,FOREIGN,0,0
4,1,DEBIT,9302.79,C1566511282,11299.0,C1973538135,29832.0,DOMESTIC,0,0


## **DATA PREPROCESSING - 3**

In [9]:
import pandas
import json
import numpy as np
import csv
import logging
from random import randint

data_path = '/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data_2.csv'

# logging.basicConfig(filename='logs/model_development.txt',
# 					filemode='a',
# 					format='%(asctime)s %(message)s',
# 					datefmt="%Y-%m-%d %H:%M:%S")

logging.warning("----------")
logging.warning("DATA PREPROCESSING 3 STAGE")

logging.warning("Reading Preprocessed 2 dataset...")

X = pandas.read_csv(data_path)
X = X.to_numpy()

logging.warning("Read Preprocessed 2 dataset")

#col
step = 0
trans_type = 1
amount = 2
nameOrig = 3
oldbalanceOrg = 4
nameDest = 5
oldbalanceDest = 6
accountType = 7
isFraud = 8

#col
entity = 0
incommingDomestic30 = 1
incommingDomestic60 = 2
incommingDomestic90 = 3
outgoingDomestic30 = 4
outgoingDomestic60 = 5
outgoingDomestic90 = 6
incommingForeign30 = 7
incommingForeign60 = 8
incommingForeign90 = 9
outgoingForeign30 = 10
outgoingForeign60 = 11
outgoingForeign90 = 12
incoming_domestic_count_30 = 13
incoming_domestic_count_60 = 14
incoming_domestic_count_90 = 15
outgoing_domestic_count_30 = 16
outgoing_domestic_count_60 = 17
outgoing_domestic_count_90 = 18
incoming_foreign_count_30 = 19
incoming_foreign_count_60 = 20
incoming_foreign_count_90 = 21
outgoing_foreign_count_30 = 22
outgoing_foreign_count_60 = 23
outgoing_foreign_count_90 = 24
balance_difference_30 = 25
balance_difference_60 = 26
balance_difference_90 = 27
isFraudSec = 28

csv_dataset_secondary = []
entities_pos = {}
enititesDict = {}

logging.warning("Creating New Features Using Transaction History...")

def getSecRow(entity):
	return [entity,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

for i in range(X.shape[0]):
	source_entity = X[i,nameOrig]
	dest_entity = X[i,nameDest]

	source_pos = entities_pos.get(source_entity,-1)
	if source_pos == -1:
		pos = len(csv_dataset_secondary)
		entities_pos[source_entity] = pos
		source_pos = pos

		row = getSecRow(source_entity)

		csv_dataset_secondary.append(row)

	dest_pos = entities_pos.get(dest_entity,-1)
	if dest_pos == -1:
		pos = len(csv_dataset_secondary)
		entities_pos[dest_entity] = pos
		dest_pos = pos

		row = getSecRow(dest_entity)

		csv_dataset_secondary.append(row)

	transferAmountSource = 0
	transferAmountDest = 0

	if X[i,trans_type] == "CASH_IN" or X[i,trans_type] == "CREDIT":
		if X[i,step] <=30:
			csv_dataset_secondary[source_pos][incommingDomestic30] += X[i,amount]
			csv_dataset_secondary[dest_pos][outgoingDomestic30] += X[i,amount]
		if X[i,step] <=60:
			csv_dataset_secondary[source_pos][incommingDomestic60] += X[i,amount]
			csv_dataset_secondary[dest_pos][outgoingDomestic60] += X[i,amount]
		if X[i,step] <=90:
			csv_dataset_secondary[source_pos][incommingDomestic90] += X[i,amount]
			csv_dataset_secondary[dest_pos][outgoingDomestic90] += X[i,amount]

		transferAmountSource = X[i,amount]
		transferAmountDest = -1*X[i,amount]

	if X[i,trans_type] == "CASH_OUT" or X[i,trans_type] == "DEBIT":
		if X[i,step] <=30:
			csv_dataset_secondary[source_pos][outgoingDomestic30] += X[i,amount]
			csv_dataset_secondary[dest_pos][incommingDomestic30] += X[i,amount]
		if X[i,step] <=60:
			csv_dataset_secondary[source_pos][outgoingDomestic60] += X[i,amount]
			csv_dataset_secondary[dest_pos][incommingDomestic60] += X[i,amount]
		if X[i,step] <=90:
			csv_dataset_secondary[source_pos][outgoingDomestic90] += X[i,amount]
			csv_dataset_secondary[dest_pos][incommingDomestic90] += X[i,amount]

		transferAmountSource = -1*X[i,amount]
		transferAmountDest = X[i,amount]

	if X[i,trans_type] == "WIRE_IN":
		if X[i,step] <=30:
			csv_dataset_secondary[source_pos][incommingForeign30] += X[i,amount]
			csv_dataset_secondary[dest_pos][outgoingForeign30] += X[i,amount]
		if X[i,step] <=60:
			csv_dataset_secondary[source_pos][incommingForeign60] += X[i,amount]
			csv_dataset_secondary[dest_pos][outgoingForeign60] += X[i,amount]
		if X[i,step] <=90:
			csv_dataset_secondary[source_pos][incommingForeign90] += X[i,amount]
			# print(dest_pos,outgoingForeign90,i,amount)
			csv_dataset_secondary[dest_pos][outgoingForeign90] += X[i,amount]

		transferAmountSource = X[i,amount]
		transferAmountDest = -1*X[i,amount]

	if X[i,trans_type] == "WIRE_OUT":
		if X[i,step] <=30:
			csv_dataset_secondary[source_pos][outgoingForeign30] += X[i,amount]
			csv_dataset_secondary[dest_pos][incommingForeign30] += X[i,amount]
		if X[i,step] <=60:
			csv_dataset_secondary[source_pos][outgoingForeign60] += X[i,amount]
			csv_dataset_secondary[dest_pos][incommingForeign60] += X[i,amount]
		if X[i,step] <=90:
			csv_dataset_secondary[source_pos][outgoingForeign90] += X[i,amount]
			csv_dataset_secondary[dest_pos][incommingForeign90] += X[i,amount]

		transferAmountSource = -1*X[i,amount]
		transferAmountDest = X[i,amount]

	if enititesDict.get(source_entity,-1) == -1:
		enititesDict[source_entity] = {
										'day1Bal': X[i,oldbalanceOrg],
										'day30Bal': 0,
										'day60Bal': 0,
										'day90Bal': 0,
										'countIncomingDomestic30': 0,
										'countOutgoingDomestic30': 0,
										'countIncomingDomestic60': 0,
										'countOutgoingDomestic60': 0,
										'countIncomingDomestic90': 0,
										'countOutgoingDomestic90': 0,
										'countIncomingForeign30': 0,
										'countOutgoingForeign30': 0,
										'countIncomingForeign60': 0,
										'countOutgoingForeign60': 0,
										'countIncomingForeign90': 0,
										'countOutgoingForeign90': 0
										}

	if enititesDict.get(dest_entity,-1) == -1:
		enititesDict[dest_entity] = {
										'day1Bal': X[i,oldbalanceDest],
										'day30Bal': 0,
										'day60Bal': 0,
										'day90Bal': 0,
										'countIncomingDomestic30': 0,
										'countOutgoingDomestic30': 0,
										'countIncomingDomestic60': 0,
										'countOutgoingDomestic60': 0,
										'countIncomingDomestic90': 0,
										'countOutgoingDomestic90': 0,
										'countIncomingForeign30': 0,
										'countOutgoingForeign30': 0,
										'countIncomingForeign60': 0,
										'countOutgoingForeign60': 0,
										'countIncomingForeign90': 0,
										'countOutgoingForeign90': 0
									}

	incomingForSource = ["CASH_IN","CREDIT","WIRE_IN"]
	incomingForDest = ["CASH_OUT","DEBIT","WIRE_OUT"]
	outgoingForDest = incomingForSource
	outgoingForSource = incomingForDest

	if X[i,step]<=30:
		enititesDict[source_entity]['day30Bal'] = transferAmountSource+X[i,oldbalanceOrg]
		enititesDict[dest_entity]['day30Bal'] = transferAmountDest+X[i,oldbalanceDest]
		if X[i,accountType] == "FOREIGN":
			if X[i,trans_type] in incomingForSource:
				enititesDict[source_entity]['countIncomingForeign30'] += 1
			else:
				enititesDict[source_entity]['countOutgoingForeign30'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[source_entity]['countIncomingDomestic30'] += 1
			else:
				enititesDict[source_entity]['countOutgoingDomestic30'] += 1


		if X[i,accountType] == "FOREIGN":
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingForeign30'] += 1
			else:

				enititesDict[dest_entity]['countOutgoingForeign30'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingDomestic30'] += 1
			else:
				enititesDict[dest_entity]['countOutgoingDomestic30'] += 1

	if X[i,step]<=60:
		enititesDict[source_entity]['day60Bal'] = transferAmountSource+X[i,oldbalanceOrg]
		enititesDict[dest_entity]['day60Bal'] = transferAmountDest+X[i,oldbalanceDest]
		if X[i,accountType] == "FOREIGN":
			if X[i,trans_type] in incomingForSource:
				enititesDict[source_entity]['countIncomingForeign60'] += 1
			else:
				enititesDict[source_entity]['countOutgoingForeign60'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[source_entity]['countIncomingDomestic60'] += 1
			else:
				enititesDict[source_entity]['countOutgoingDomestic60'] += 1

		if X[i,accountType] == "FOREIGN":
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingForeign60'] += 1
			else:
				enititesDict[dest_entity]['countOutgoingForeign60'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingDomestic60'] += 1
			else:
				enititesDict[dest_entity]['countOutgoingDomestic60'] += 1

	if X[i,step]<=90:
		enititesDict[source_entity]['day90Bal'] = transferAmountSource+X[i,oldbalanceOrg]
		enititesDict[dest_entity]['day90Bal'] = transferAmountDest+X[i,oldbalanceDest]
		if X[i,accountType] == "FOREIGN":
			if X[i,trans_type] in incomingForSource:
				enititesDict[source_entity]['countIncomingForeign90'] += 1
			else:
				enititesDict[source_entity]['countOutgoingForeign90'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[source_entity]['countIncomingDomestic90'] += 1
			else:
				enititesDict[source_entity]['countOutgoingDomestic90'] += 1

		if X[i,accountType] == "FOREIGN":
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingForeign90'] += 1
			else:
				enititesDict[dest_entity]['countOutgoingForeign90'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingDomestic90'] += 1
			else:
				enititesDict[dest_entity]['countOutgoingDomestic90'] += 1


	csv_dataset_secondary[source_pos][balance_difference_30] = enititesDict[source_entity]['day30Bal'] - enititesDict[source_entity]['day1Bal']
	csv_dataset_secondary[source_pos][balance_difference_60] = enititesDict[source_entity]['day60Bal'] - enititesDict[source_entity]['day1Bal']
	csv_dataset_secondary[source_pos][balance_difference_90] = enititesDict[source_entity]['day90Bal'] - enititesDict[source_entity]['day1Bal']



	csv_dataset_secondary[source_pos][incoming_domestic_count_30] = enititesDict[source_entity]['countIncomingDomestic30']
	csv_dataset_secondary[source_pos][outgoing_domestic_count_30] = enititesDict[source_entity]['countOutgoingDomestic30']
	csv_dataset_secondary[source_pos][incoming_domestic_count_60] = enititesDict[source_entity]['countIncomingDomestic60']
	csv_dataset_secondary[source_pos][outgoing_domestic_count_60] = enititesDict[source_entity]['countOutgoingDomestic60']
	csv_dataset_secondary[source_pos][incoming_domestic_count_90] = enititesDict[source_entity]['countIncomingDomestic90']
	csv_dataset_secondary[source_pos][outgoing_domestic_count_90] = enititesDict[source_entity]['countOutgoingDomestic90']
	csv_dataset_secondary[source_pos][incoming_foreign_count_30] = enititesDict[source_entity]['countIncomingForeign30']
	csv_dataset_secondary[source_pos][outgoing_foreign_count_30] = enititesDict[source_entity]['countOutgoingForeign30']
	csv_dataset_secondary[source_pos][incoming_foreign_count_60] = enititesDict[source_entity]['countIncomingForeign60']
	csv_dataset_secondary[source_pos][outgoing_foreign_count_60] = enititesDict[source_entity]['countOutgoingForeign60']
	csv_dataset_secondary[source_pos][incoming_foreign_count_90] = enititesDict[source_entity]['countIncomingForeign90']
	csv_dataset_secondary[source_pos][outgoing_foreign_count_90] = enititesDict[source_entity]['countOutgoingForeign90']

	csv_dataset_secondary[source_pos][isFraudSec] = csv_dataset_secondary[source_pos][isFraudSec] or X[i,isFraud]

	csv_dataset_secondary[dest_pos][incoming_domestic_count_30] = enititesDict[dest_entity]['countIncomingDomestic30']
	csv_dataset_secondary[dest_pos][outgoing_domestic_count_30] = enititesDict[dest_entity]['countOutgoingDomestic30']
	csv_dataset_secondary[dest_pos][incoming_domestic_count_60] = enititesDict[dest_entity]['countIncomingDomestic60']
	csv_dataset_secondary[dest_pos][outgoing_domestic_count_60] = enititesDict[dest_entity]['countOutgoingDomestic60']
	csv_dataset_secondary[dest_pos][incoming_domestic_count_90] = enititesDict[dest_entity]['countIncomingDomestic90']
	csv_dataset_secondary[dest_pos][outgoing_domestic_count_90] = enititesDict[dest_entity]['countOutgoingDomestic90']
	csv_dataset_secondary[dest_pos][incoming_foreign_count_30] = enititesDict[dest_entity]['countIncomingForeign30']
	csv_dataset_secondary[dest_pos][outgoing_foreign_count_30] = enititesDict[dest_entity]['countOutgoingForeign30']
	csv_dataset_secondary[dest_pos][incoming_foreign_count_60] = enititesDict[dest_entity]['countIncomingForeign60']
	csv_dataset_secondary[dest_pos][outgoing_foreign_count_60] = enititesDict[dest_entity]['countOutgoingForeign60']
	csv_dataset_secondary[dest_pos][incoming_foreign_count_90] = enititesDict[dest_entity]['countIncomingForeign90']
	csv_dataset_secondary[dest_pos][outgoing_foreign_count_90] = enititesDict[dest_entity]['countOutgoingForeign90']


columns = ['entity','incoming_domestic_amount_30','incoming_domestic_amount_60','incoming_domestic_amount_90',
                     'outgoing_domestic_amount_30','outgoing_domestic_amount_60','outgoing_domestic_amount_90',
                     'incoming_foreign_amount_30','incoming_foreign_amount_60','incoming_foreign_amount_90',
                     'outgoing_foreign_amount_30','outgoing_foreign_amount_60','outgoing_foreign_amount_90',
                     'incoming_domestic_count_30','incoming_domestic_count_60','incoming_domestic_count_90',
                     'outgoing_domestic_count_30','outgoing_domestic_count_60','outgoing_domestic_count_90',
                     'incoming_foreign_count_30','incoming_foreign_count_60','incoming_foreign_count_90',
                     'outgoing_foreign_count_30','outgoing_foreign_count_60','outgoing_foreign_count_90',
                     'balance_difference_30','balance_difference_60','balance_difference_90','isFraud']

logging.warning("Creating New Features Done")

logging.warning("Storing Data in Data_processed Folder...")

data_secondary = pandas.DataFrame(csv_dataset_secondary, columns=columns)
data_secondary.to_csv('/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data_3.csv', index=False)

logging.warning("Storing Data Done")


In [10]:
data_path = "/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data_3.csv"
filtereddata = pd.read_csv(data_path)
filtereddata.head()

,entity,incoming_domestic_amount_30,incoming_domestic_amount_60,incoming_domestic_amount_90,outgoing_domestic_amount_30,outgoing_domestic_amount_60,outgoing_domestic_amount_90,incoming_foreign_amount_30,incoming_foreign_amount_60,incoming_foreign_amount_90,...,incoming_foreign_count_30,incoming_foreign_count_60,incoming_foreign_count_90,outgoing_foreign_count_30,outgoing_foreign_count_60,outgoing_foreign_count_90,balance_difference_30,balance_difference_60,balance_difference_90,isFraud
0,C840083671,0.00,0.00,0.00,181.00,181.00,181.00,0.00,0.00,0.00,...,0,0,0,0,0,0,-181.00,-181.00,-181.00,1
1,C38997010,1882856.15,2726258.90,2726258.90,625257.04,835185.12,835185.12,967120.80,967120.80,967120.80,...,1,1,1,1,3,3,0.00,0.00,0.00,0
2,C1900366749,0.00,0.00,0.00,9644.94,9644.94,9644.94,0.00,0.00,0.00,...,0,0,0,0,0,0,-9644.94,-9644.94,-9644.94,0
3,C997608398,3428167.79,3598695.35,3598695.35,2787499.21,3047168.93,3047168.93,1359022.86,1359022.86,1359022.86,...,3,3,3,6,6,6,0.00,0.00,0.00,0
4,C905080434,0.00,0.00,0.00,229133.94,229133.94,229133.94,0.00,0.00,0.00,...,0,0,0,0,0,0,-229133.94,-229133.94,-229133.94,0


# **FEATURE SELECTION**

In [11]:
import numpy as np
import math
from numpy import genfromtxt
import pandas
import logging
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# logging.basicConfig(filename='logs/model_development.txt',
# 					filemode='a',
# 					format='%(asctime)s %(message)s',
# 					datefmt="%Y-%m-%d %H:%M:%S")

logging.warning("----------")
logging.warning("FEATURE SELECTION STAGE")

logging.warning("Reading Filtered Data 3 ...")

dataframeX = pandas.read_csv('/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data_3.csv')
col_names = list(dataframeX.columns.values)
dataMat = dataframeX.to_numpy()

logging.warning("Read Filtered Data 3")

logging.warning("Creating X and Y Variables...")

X = dataMat[:,1:-2]
Y = dataMat[:,-1]

logging.warning(f"Shape of X: {X.shape} and Shape of Y: {Y.shape}")

logging.warning("Instiantiating Random Forest Model...")

model = RandomForestClassifier(random_state=42)

logging.warning("Fitting Data...")

model.fit(X, Y.astype(int))

logging.warning("Checking Feature Importances...")

feature_imp = model.feature_importances_

sorted_feature_vals = np.sort(feature_imp)
sorted_feature_indexes = np.argsort(feature_imp)

logging.warning("Significant Features in decreasing order of importance: ")

logging.warning("Storing Feature Importances in reports...")

fea_imp = [[col_names[i+2], feature_imp[i]] for i in reversed(sorted_feature_indexes)]
features = pandas.DataFrame(fea_imp, columns=["features", "importance_score"])
features.to_csv("/content/drive/MyDrive/miniproject 2/project/reports/feature_importances.csv", index=False)

logging.warning("Storing Feature Importances Done")



In [12]:
data_path = "/content/drive/MyDrive/miniproject 2/project/reports/feature_importances.csv"
filtereddata = pd.read_csv(data_path)
filtereddata.head()

,features,importance_score
0,balance_difference_90,0.260016
1,balance_difference_60,0.256726
2,incoming_foreign_amount_30,0.150099
3,outgoing_domestic_amount_90,0.110576
4,outgoing_domestic_amount_60,0.085944


Segment Generator

In [13]:
import time
import json
import logging
import pandas
import numpy as np
import csv
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# logging.basicConfig(filename='logs/model_development.txt',
# 					filemode='a',
# 					format='%(asctime)s %(message)s',
# 					datefmt="%Y-%m-%d %H:%M:%S")

logging.warning("----------")
logging.warning("SEGMENT GENERATOR STAGE")

def getClusterPredictions(data, true_k):
	model = KMeans(n_clusters=true_k)
	model.fit(data)
	prediction = model.predict(data)

	return prediction

def getBestCluster(X,_min=2,_max=10):
	selected_cluster = 0
	previous_sil_coeff = 0.001 #some random small number not 0
	sc_vals = []
	for n_cluster in range(_min, _max):
	    kmeans = KMeans(n_clusters=n_cluster).fit(X)
	    label = kmeans.labels_

	    sil_coeff = silhouette_score(X, label, metric='euclidean', sample_size=1000)
	    sc_vals.append(sil_coeff)
	    # print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))

	    percent_change = (sil_coeff-previous_sil_coeff)*100/previous_sil_coeff

	    # return when below a threshold of 1%
	    if percent_change<1:
	    	selected_cluster = n_cluster-1

	    previous_sil_coeff = sil_coeff

	return selected_cluster or _max, sc_vals

data_path = "/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data_3.csv"

logging.warning("Reading Filtered Data 3 ...")

X_dataframe = pandas.read_csv(data_path)
X = X_dataframe.to_numpy()

logging.warning("Read Filtered Data 3")

col_names = list(X_dataframe.columns.values)

X_trimmed_features = np.zeros((X.shape[0],1))

logging.warning("Importing Feature Importances...")

feature_path = "/content/drive/MyDrive/miniproject 2/project/reports/feature_importances.csv"
features = pandas.read_csv(feature_path)

logging.warning("Selecting Top 13 Features for CLustering...")

top_13 = features.iloc[:13, 0].tolist()

logging.warning("Top 13 Features stored in List")

for feature in top_13:
	X_trimmed_features = np.concatenate((X_trimmed_features,np.expand_dims(X_dataframe[feature],axis=1)),axis=1)
X_trimmed_features = X_trimmed_features[:,1:]

logging.warning("Choosing Best Number Of Clusters...")

min_value = 2
max_value = 10
true_k, sc_vals = getBestCluster(X_trimmed_features,_min=min_value,_max=max_value)
true_k = 5

logging.warning("Storing Silhoutte Scores...")


sil_score = [[i, sc_vals[i-min_value]] for i in range(min_value, max_value)]
sil = pandas.DataFrame(sil_score, columns=["no_of_clusters", "silhoutte_score"])
sil.to_csv("/content/drive/MyDrive/project/reports/silhoutte_scores.csv", index=False)

logging.warning("Storing Silhoutte Scores Done")

logging.warning("Creating Clusters with Best No Of Clusters...")

prediction = getClusterPredictions(X_trimmed_features, true_k)
seg_dict = {}
for i in range(X.shape[0]):
	seg_dict[X[i,0]] = prediction[i]

logging.warning("Inputing Filtered Data 2 Dataset...")

filter_1_path = "/content/drive/MyDrive/miniproject 2/project/data_processed/filtered_data_2.csv"
X_dataframe_pri = pandas.read_csv(filter_1_path)
X_pri = X_dataframe_pri.to_numpy()
col_names = list(X_dataframe_pri.columns.values)

logging.warning("Read Filtered 2 Data")

logging.warning("Creating Final Dataset with segments...")

X_with_segments = []
for i in range(X_pri.shape[0]):
		X_with_segments.append(np.concatenate(([[seg_dict[X_pri[i,3]]]],np.expand_dims(X_pri[i,:],axis=0)),axis=1)[0])

segmented_columns = ['segment','step','trans_type','amount','nameOrig','oldbalanceOrg',
                    'nameDest','oldbalanceDest','accountType','isFraud','isFlaggedFraud']

data_segmented = pandas.DataFrame(X_with_segments, columns = segmented_columns)
data_segmented = data_segmented.drop('isFlaggedFraud', axis=1)
data_segmented.to_csv('/content/drive/MyDrive/miniproject 2/project/data_processed/final_data.csv', index=False)

logging.warning("Storing Final Dataset Done")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [14]:
data_path = "/content/drive/MyDrive/miniproject 2/project/data_processed/final_data.csv"
filtereddata = pd.read_csv(data_path)
filtereddata.head()

,segment,step,trans_type,amount,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest,accountType,isFraud
0,0,1,CASH_OUT,181.00,C840083671,181.0,C38997010,21182.0,DOMESTIC,1
1,0,1,DEBIT,9644.94,C1900366749,4465.0,C997608398,10845.0,DOMESTIC,0
2,0,1,CASH_OUT,229133.94,C905080434,15325.0,C476402209,5083.0,DOMESTIC,0
3,0,1,WIRE_IN,215310.30,C1670993182,705.0,C1100439041,22425.0,FOREIGN,0
4,0,1,DEBIT,9302.79,C1566511282,11299.0,C1973538135,29832.0,DOMESTIC,0


In [15]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 11.6 MB/s eta 0:00:00


# **MODEL CREATION**

In [16]:
import numpy as np
import pandas as pd
import pickle
import json
from datetime import datetime
import matplotlib.pyplot as plt
import logging
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, accuracy_score
from catboost import CatBoostClassifier

# logging.basicConfig(filename='logs/model_development.txt',
# 					filemode='a',
# 					format='%(asctime)s %(message)s',
# 					datefmt="%Y-%m-%d %H:%M:%S")

logging.warning("----------")
logging.warning("MODEL CREATION STAGE")

logging.warning("Reading Final Dataset...")

data_path = "/content/drive/MyDrive/miniproject 2/project/data_processed/final_data.csv"
dataMat = pd.read_csv(data_path)
data = dataMat.to_numpy()

logging.warning("Read Final Dataset")

logging.warning("Checking Categorical Features...")

cat_feat = [i for i in dataMat.columns if dataMat[i].dtypes == 'O']

logging.warning("Checking Missing Values...")

a = dict(dataMat.isnull().sum())
b = [[i, a[i]] for i in a.keys()]
missing = pd.DataFrame(b, columns=['features', 'null_values_count'])

logging.warning("Storing Missing Values...")

missing.to_csv("/content/drive/MyDrive/miniproject 2/project/reports/missing_values.csv", index=False)

logging.warning("Storing Missing Values Done")

logging.warning("Encoding Categorical Features...")

encoder = LabelEncoder()
for i in cat_feat:
  dataMat[i] = encoder.fit_transform(dataMat[i])

logging.warning("Features Encoding Done")

logging.warning("Creating X and y variables ...")

X = dataMat.iloc[:, :-1]
y = dataMat['isFraud']
X.to_csv("/content/drive/MyDrive/miniproject 2/project/input.csv",index=False)
logging.warning(f"Shape of X: {X.shape} and Shape of y: {y.shape}")

logging.warning("Splitting Dataset...")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

logging.warning("Instantiating Model...")

model = CatBoostClassifier(random_state=42, class_weights={0:1, 1:12}, silent=True)

logging.warning("Fitting Model...")

model.fit(X_train, y_train)
y_pred_cat = model.predict(X_test)

logging.warning("Saving Model...")

model_path = "/content/drive/MyDrive/miniproject 2/project/saved_models/model.pkl"
pickle.dump(model, open(model_path, 'wb'))

logging.warning("Saving Model Metrics...")









In [17]:
model_metric = {
    "time_stamp": datetime.now().strftime("%d-%m-%Y_%H:%M:%S"),
    "confusion_matrix": confusion_matrix(y_test, y_pred_cat).tolist(),
    "precision": precision_score(y_test, y_pred_cat),
    "recall": recall_score(y_test, y_pred_cat),
    "f1_score": f1_score(y_test, y_pred_cat),
    "accuracy" : accuracy_score(y_test, y_pred_cat)
}


#output of model metrices
print("Model Metrices")
print("Time Stamp")
print(datetime.now().strftime("%d-%m-%Y_%H:%M:%S"))
print("Confusion Matrix")
print( confusion_matrix(y_test, y_pred_cat))
print("Precision Score")
print( precision_score(y_test, y_pred_cat))
print("Recall Score")
print(recall_score(y_test, y_pred_cat))
print("F1 Score")
print(f1_score(y_test, y_pred_cat))
print("Accuracy")
print(accuracy_score(y_test, y_pred_cat))

Model Metrices
Time Stamp
15-10-2023_11:17:49
Confusion Matrix
[[63912     6]
 [   29    12]]
Precision Score
0.6666666666666666
Recall Score
0.2926829268292683
F1 Score
0.4067796610169492
Accuracy
0.9994527744336216


# **PREDICTION PART**


In [19]:
outputpath = "/content/drive/MyDrive/miniproject 2/project/input.csv"
outputdata = pandas.read_csv(outputpath)
outputdata.head()

,segment,step,trans_type,amount,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest,accountType
0,0,1,1,181.00,195473,181.0,3045,21182.0,0
1,0,1,3,9644.94,98690,4465.0,4488,10845.0,0
2,0,1,1,229133.94,202733,15325.0,3262,5083.0,0
3,0,1,4,215310.30,73648,705.0,222,22425.0,1
4,0,1,3,9302.79,62140,11299.0,2281,29832.0,0


In [20]:
from flask import Flask, request, render_template
from flask import Response
from datetime import datetime
import pandas
import pickle
import json
import requests


model_path = '/content/drive/MyDrive/miniproject 2/project/saved_models/model.pkl'
model = pickle.load(open(model_path, 'rb'))

path = "/content/drive/MyDrive/miniproject 2/project/input.csv"
data = pandas.read_csv(path)
y_pred = model.predict(data)
data['isFraud'] = y_pred
output_path = "/content/drive/MyDrive/miniproject 2/project/Output.csv"
print(data)
data.to_csv(output_path, index=False)

        segment  step  trans_type     amount  nameOrig  oldbalanceOrg  \
0             0     1           1     181.00    195473          181.0   
1             0     1           3    9644.94     98690         4465.0   
2             0     1           1  229133.94    202733        15325.0   
3             0     1           4  215310.30     73648          705.0   
4             0     1           3    9302.79     62140        11299.0   
...         ...   ...         ...        ...       ...            ...   
213190        0   717           1  136062.35    173885        50150.0   
213191        0   717           0   36042.63     34172        32972.0   
213192        0   717           0  183079.23    118885       106826.0   
213193        0   718           0  154599.28    167799        59768.0   
213194        0   718           3    1636.03    186682        83120.0   

        nameDest  oldbalanceDest  accountType  isFraud  
0           3045        21182.00            0        0  
1        

In [21]:
outputpath = "/content/drive/MyDrive/miniproject 2/project/Output.csv"
outputdata = pandas.read_csv(outputpath)
outputdata.head()

,segment,step,trans_type,amount,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest,accountType,isFraud
0,0,1,1,181.00,195473,181.0,3045,21182.0,0,0
1,0,1,3,9644.94,98690,4465.0,4488,10845.0,0,0
2,0,1,1,229133.94,202733,15325.0,3262,5083.0,0,0
3,0,1,4,215310.30,73648,705.0,222,22425.0,1,0
4,0,1,3,9302.79,62140,11299.0,2281,29832.0,0,0


In [22]:
df =outputdata[outputdata['isFraud']==1]
print(df.head())
print(df.tail())

      segment  step  trans_type      amount  nameOrig  oldbalanceOrg  \
11          0     1           1    23261.30    117730       20411.53   
448         0     1           1  1277212.77    154030     1277212.77   
480         0     1           1    35063.63     69772       35063.63   
794         0     1           1   132842.64     40272        4499.08   
1219        0     2           1  1096187.24    187823     1096187.24   

      nameDest  oldbalanceDest  accountType  isFraud  
11        2331         25742.0            0        1  
448       3838             0.0            0        1  
480       2302         31140.0            0        1  
794       2857             0.0            0        1  
1219      3657             0.0            0        1  
        segment  step  trans_type      amount  nameOrig  oldbalanceOrg  \
211275        0   524           1   232315.43    102763      232315.43   
211794        0   551           1   813992.49    212186      813992.49   
211914        0

In [23]:
print(len(df))
print(len(data))

99
213195


In [24]:
#fraud data to csv
Output_Fraud_path = "/content/drive/MyDrive/miniproject 2/project/isFraudTrue.csv"
df.to_csv(Output_Fraud_path , index=False)

In [25]:
data_path = "/content/drive/MyDrive/miniproject 2/project/isFraudTrue.csv"
filtereddata = pd.read_csv(data_path)
filtereddata.head()

,segment,step,trans_type,amount,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest,accountType,isFraud
0,0,1,1,23261.30,117730,20411.53,2331,25742.0,0,1
1,0,1,1,1277212.77,154030,1277212.77,3838,0.0,0,1
2,0,1,1,35063.63,69772,35063.63,2302,31140.0,0,1
3,0,1,1,132842.64,40272,4499.08,2857,0.0,0,1
4,0,2,1,1096187.24,187823,1096187.24,3657,0.0,0,1
